In [ ]:
from lxml import etree
import numpy as np
import os
import pandas as pd
# import requests
import urllib.request
import yaml

# Parse OTE Car # and APT Program # from PIF Summary

"PIF Status Summary" file can be downloaded from Box: https://stsci.app.box.com/s/etmxlxpvsneiede1dc5hhft2hjwelels/folder/3346031526

In [ ]:
# Parse PIF summary Excel file into pandas dataframe
pif_summary_xlsx = '/Users/lchambers/TEL/Commissioning/PIF Status Summary 2019-01.xlsx'
pif_summary = pd.read_excel(pif_summary_xlsx)
pif_summary.columns

In [ ]:
# Look at columns for OTE # and APT program #
pif_summary[['OTE #', 'APT Prog. ID']]

In [ ]:
# Start to build commissioning dictionary
commissioning_dict = {}
for index, row in pif_summary.iterrows():
    car = row['OTE #']
    apt = row['APT Prog. ID']
#     print(row['APT Prog. ID'], type(row['APT Prog. ID']), pd.isna(row['APT Prog. ID']))
    if pd.notna(car) and pd.notna(apt) and \
        isinstance(apt, int) and len(car.split()) == 1:
        
        commissioning_dict[car.lower()] = {}
        commissioning_dict[car.lower()]['apt'] = apt
        
#         print(row['OTE #'], row['APT Prog. ID'], type(row['APT Prog. ID']))

commissioning_dict

# Download each APT program and get # of observations

In [ ]:
# List all APT programs
all_apt_programs = [commissioning_dict[item]['apt'] for item in commissioning_dict.keys()]
all_cars = list(commissioning_dict.keys())

# Build temporary directory
!mkdir temp_apt

# For each APT program...
for prog, car in zip(all_apt_programs, all_cars):
    print('''* * * * * *
STARTING PROGRAM {}
* * * * * *
    '''.format(prog))
    
    # Download the APT file
    urllib.request.urlretrieve(
        'http://www.stsci.edu/jwst/phase2-public/{}.aptx'.format(prog), 
        'temp_apt/{}.aptx'.format(prog)
    )
    
    # Export the APT XML file
    os.system(
        "/Applications/APT\ 26.2.2/bin/apt -nogui -export xml temp_apt/{}.aptx"
        .format(prog)
    )
    
    # Figure out how many observations are in the program
    apt_namespace = '{http://www.stsci.edu/JWST/APT}'
    with open('temp_apt/{}.xml'.format(prog)) as f:
        tree = etree.parse(f)
    observation_data = tree.find(apt_namespace + 'DataRequests')
    observation_list = observation_data.findall('.//' + apt_namespace + 'Observation')
    n_obs = len(observation_list)
    
    # Add to the commissioning directory
    commissioning_dict[car]['observations'] = n_obs
    
# Tear down temporary directory
!rm -r temp_apt

# Save out file

In [ ]:
commissioning_dict

In [ ]:
commissioning_activities_yaml = '/Users/lchambers/TEL/Commissioning/tools/fgs-commissioning/jwst_magic/data/commissioning_activities.yaml'

with open(commissioning_activities_yaml, 'w') as outfile:
    yaml.dump(commissioning_dict, outfile, default_flow_style=False) 